In [1]:
# libraries
import numpy as np
import pandas as pd
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import add_dummy_feature
alt.data_transformers.disable_max_rows()
import warnings
warnings.filterwarnings('ignore')

In [6]:
# load tidied data and print rows
data_2021 = pd.read_excel('Tidy Data/data_2021.xlsx')
data_2022 = pd.read_excel('Tidy Data/data_2022.xlsx')

In [7]:
# First we select data that has two years records
common = set(np.unique(data_2021.state)) & set(np.unique(data_2022.state)) # Get common states

mask = data_2021.state.isin(common)
data_2021 = data_2021[mask]

mask1 = data_2022.state.isin(common)
data_2022 = data_2022[mask1]


# Now get a new dataframe consists the sum of records at each state for 2021
data_2021_sum = data_2021.groupby(['state', 'year']).sum().reset_index().drop(columns = ['Unnamed: 0', 'month'])

# Subtract the first month of each state by the sum in 2021
data_merged = pd.merge(data_2021_sum, data_2022, on='state')
data_pivot = data_merged.pivot(index='state', columns='month', values= ['total_vaccinations_x','total_distributed_x','people_vaccinated_x','total_vaccinations_y','total_distributed_y','people_vaccinated_y'])

total_vaccinations_Jan = ((data_pivot['total_vaccinations_y'][1] - data_pivot['total_vaccinations_x'][1]))
total_distributed_Jan = data_pivot['total_distributed_y'][1] - data_pivot['total_distributed_x'][1]
people_vaccinated_Jan = data_pivot['people_vaccinated_y'][1] - data_pivot['people_vaccinated_x'][1]

# Now replace the original data
for i in np.unique(data_2022.state):
    if ((data_2022.state == i) & (data_2022.month == 1)).any():
        data_2022.loc[(data_2022['state'] == i) & (data_2022['month'] == 1), 'total_vaccinations'] = total_vaccinations_Jan.loc[i]
        data_2022.loc[(data_2022['state'] == i) & (data_2022['month'] == 1), 'total_distributed'] = total_distributed_Jan.loc[i]
        data_2022.loc[(data_2022['state'] == i) & (data_2022['month'] == 1), 'people_vaccinated'] = people_vaccinated_Jan.loc[i]


# Question: Is there a relationship between the more-than-one-dose rate and death rate of the states?

### Calculating rates

#### 2021:

In [8]:
# More than one dose rate
data_2021['multiple_dose_rate'] = (data_2021['total_vaccinations'] - data_2021['people_vaccinated']) / data_2021['people_vaccinated']

# Death rate
data_2021['death_rate'] = data_2021['new_deaths'] / data_2021['new_cases']

#### 2022:

In [9]:
# More than one dose rate
data_2022['multiple_dose_rate'] = (data_2022['total_vaccinations'] - data_2022['people_vaccinated']) / data_2022['people_vaccinated']

# Death rate
data_2022['death_rate'] = data_2022['new_deaths'] / data_2022['new_cases']

### EDA for multiple dose and vaccination rates

In [10]:
data_2021[data_2021.multiple_dose_rate < 0]

,Unnamed: 0,year,month,state,new_cases,new_deaths,total_vaccinations,total_distributed,people_vaccinated,multiple_dose_rate,death_rate
2,2,2021,3,Alabama,22504,740,353974,1394350,505394,-0.299608,0.032883
3,3,2021,4,Alabama,14416,395,353974,1291400,434379,-0.185103,0.027400


In [11]:
data_2021 = data_2021.drop(data_2021.loc[data_2021.multiple_dose_rate < 0,:].index, axis = 0)

#### Plotting Death Rate vs Multiple Dose Rate

In [12]:
plt_2021_rate = alt.Chart(data_2021).mark_circle(color = 'salmon', size = 30).encode(
    x = alt.X('multiple_dose_rate', title = 'Multiple dose rate', scale = alt.Scale(zero = False, type = 'pow', exponent = 1/4)),
    y = alt.Y('death_rate', title = 'Death rate', scale = alt.Scale(zero = False, type = 'pow', exponent = 1/2))
)
plt_2021_rate

alt.Chart(...)

In [13]:
plt_2022_rate = alt.Chart(data_2022).mark_circle(color = 'lightgreen', size = 30).encode(
    x = alt.X('multiple_dose_rate', title = 'Multiple dose rate', scale = alt.Scale(zero = False, type = 'pow', exponent = 1/4)),
    y = alt.Y('death_rate', title = 'Death rate', scale = alt.Scale(zero = False, type = 'pow', exponent = 1/2))
)
plt_2022_rate

alt.Chart(...)

The multiple dose rate plots look very similar to the vaccination plots for our first questions EDA

## Fitting Linear Models

### 2021:

In [14]:
# store response variable as array
y = data_2021.death_rate

# store explanatory variable
x_slr_df = data_2021.loc[:,['multiple_dose_rate']]

# add intercept column
x_slr = add_dummy_feature(x_slr_df, value = 1)

# print first five rows
x_slr[0:5, :]

array([[1.        , 0.18663364],
       [1.        , 0.00719887],
       [1.        , 1.00509808],
       [1.        , 0.8817294 ],
       [1.        , 1.17324623]])

#### Model Fitting

In [15]:
# configure module
slr = LinearRegression(fit_intercept = False)

# fit model
slr.fit(x_slr, y)

LinearRegression(fit_intercept=False)

#### Retrieve estimated coefficients

In [16]:
# store the coefficient estimates
coef = slr.coef_

# set beta values
b0 = coef[0]
b1 = coef[1]

#### Fitted values and residuals

In [17]:
# fitted values
fitted_slr = slr.predict(x_slr)

fitted_slr[0:5]

array([0.0165392 , 0.01653674, 0.01655044, 0.01654874, 0.01655275])

#### Compute Residuals

In [18]:
# residuals
resid_slr = y-fitted_slr

#print
resid_slr

0      0.009418
1      0.046076
4     -0.000289
5      0.011770
6     -0.009783
         ...   
595   -0.012994
596   -0.011191
597   -0.009405
598   -0.010179
599   -0.008095
Name: death_rate, Length: 502, dtype: float64

#### Append fitted values and residuals to the data

In [19]:
# append
data_2021['fitted_slr'] = fitted_slr
data_2021['resid_slr'] = resid_slr

# print
data_2021.head()

,Unnamed: 0,year,month,state,new_cases,new_deaths,total_vaccinations,total_distributed,people_vaccinated,multiple_dose_rate,death_rate,fitted_slr,resid_slr
0,0,2021,1,Alabama,92267,2395,353974,659400,298301,0.186634,0.025957,0.016539,0.009418
1,1,2021,2,Alabama,41078,2572,353974,742880,351444,0.007199,0.062613,0.016537,0.046076
4,4,2021,5,Alabama,15743,256,353974,490560,176537,1.005098,0.016261,0.016550,-0.000289
5,5,2021,6,Alabama,6568,186,353974,234460,188111,0.881729,0.028319,0.016549,0.011770
6,6,2021,7,Alabama,28067,190,353974,346840,162878,1.173246,0.006770,0.016553,-0.009783


In [20]:
# store n (number of observations) and p (number of betas)
n, p = x_slr.shape

# compute estimate of error variance
sigma2_hat = ((n - 1)/(n - p)) * resid_slr.var()

In [21]:
# compute X'X
xtx_slr = x_slr.transpose().dot(x_slr)

# compute matrix of parameter variances/covariances: estimated error variance x (X'X)^{-1}
slrcoef_vcov = np.linalg.inv(xtx_slr) * sigma2_hat

# take square root of matrix diagonals to get standard errors 
slrcoef_se = np.sqrt(slrcoef_vcov.diagonal())

# print
slrcoef_se

array([0.00110667, 0.00049111])

In [22]:
# create table
slrcoef_table = pd.DataFrame(
    data = {'coefficient estimate': slr.coef_, 'standard error': slrcoef_se},
    index= ['intercept', 'Multiple Dose rate']
)

# print
slrcoef_table

,coefficient estimate,standard error
intercept,0.016537,0.001107
Multiple Dose rate,0.000014,0.000491


#### R^2

In [23]:
# compute R-squared
r2_score(y, data_2021.fitted_slr)

1.5628803828882099e-06

The R^2 value is near 0, which suggests that multiple dose rate does not predict the death rate in 2021.

### 2022:

In [24]:
# store response variable as array
y2 = data_2022.death_rate

# store explanatory variable
x_slr_df2 = data_2022.loc[:,['multiple_dose_rate']]

# add intercept column
x_slr2 = add_dummy_feature(x_slr_df2, value = 1)

# print first five rows
x_slr2[0:5, :]

array([[ 1.        , 12.02818755],
       [ 1.        ,  3.00472745],
       [ 1.        ,  2.38263649],
       [ 1.        ,  4.58635125],
       [ 1.        ,  4.84046421]])

#### Model Fitting

In [25]:
# configure module
slr2 = LinearRegression(fit_intercept = False)

# fit model
slr2.fit(x_slr2, y2)

LinearRegression(fit_intercept=False)

#### Retrieve estimated coefficients

In [26]:
# fitted values
fitted_slr2 = slr2.predict(x_slr2)

fitted_slr2[0:5]

array([0.00902993, 0.011451  , 0.01161792, 0.01102664, 0.01095846])

#### Fitted values and residuals

In [27]:
# fitted values
fitted_slr2 = slr2.predict(x_slr2)

fitted_slr2[0:5]

array([0.00902993, 0.011451  , 0.01161792, 0.01102664, 0.01095846])

#### Compute Residuals

In [28]:
# residuals
resid_slr2 = y2-fitted_slr2

#print
resid_slr2

0     -0.007393
1     -0.001824
2      0.053394
3      0.041075
4     -0.001883
         ...   
511   -0.008554
512   -0.004266
513   -0.002322
514   -0.002831
515   -0.003920
Name: death_rate, Length: 504, dtype: float64

#### Append fitted values and residuals to the data

In [29]:
# append
data_2022['fitted_slr'] = fitted_slr2
data_2022['resid_slr'] = resid_slr2

# print
data_2022.head()

,Unnamed: 0,year,month,state,new_cases,new_deaths,total_vaccinations,total_distributed,people_vaccinated,multiple_dose_rate,death_rate,fitted_slr,resid_slr
0,0,2022,1,Alabama,281623,461,1712438,845870,131441,12.028188,0.001637,0.009030,-0.007393
1,1,2022,2,Alabama,113427,1092,138928,356640,34691,3.004727,0.009627,0.011451,-0.001824
2,2,2022,3,Alabama,19658,1278,64561,162300,19086,2.382636,0.065012,0.011618,0.053394
3,3,2022,4,Alabama,5259,274,114766,283900,20544,4.586351,0.052101,0.011027,0.041075
4,4,2022,5,Alabama,10247,93,83542,218200,14304,4.840464,0.009076,0.010958,-0.001883


In [30]:
# store n (number of observations) and p (number of betas)
n2, p2 = x_slr2.shape

# compute estimate of error variance
sigma2_hat2 = ((n2 - 1)/(n2 - p2)) * resid_slr2.var()

In [31]:
# compute X'X
xtx_slr2 = x_slr2.transpose().dot(x_slr2)

# compute matrix of parameter variances/covariances: estimated error variance x (X'X)^{-1}
slrcoef_vcov2 = np.linalg.inv(xtx_slr2) * sigma2_hat2

# take square root of matrix diagonals to get standard errors 
slrcoef_se2 = np.sqrt(slrcoef_vcov2.diagonal())

# print
slrcoef_se2

array([0.00120466, 0.00017311])

In [32]:
# create table
slrcoef_table2 = pd.DataFrame(
    data = {'coefficient estimate': slr2.coef_, 'standard error': slrcoef_se2},
    index= ['intercept', 'Multiple Dose Rate']
)

# print
slrcoef_table2

,coefficient estimate,standard error
intercept,0.012257,0.001205
Multiple Dose Rate,-0.000268,0.000173


In [33]:
# compute R-squared
r2_score(y2, data_2022.fitted_slr)

0.004762891696651228

The R^2 value is near 0, which suggests that multiple dose rate does not predict the death rate in 2022.